In [1]:
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime
from loguru import logger
from pathlib import Path
import numpy as np
import spatialdata as sd
from napari_spatialdata import Interactive

from multiplex_pipeline.utils.config_loaders import load_analysis_settings
from multiplex_pipeline.processors import build_processor
from multiplex_pipeline.processors.controller import ResourceBuildingController

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


### Load analysis settings

In [2]:
# load analysis configuration
settings_path = r'C:\sdata_NRR-W\analysis_settings_NRR-W.yaml'

overwrite_mask = True

settings = load_analysis_settings(settings_path)

### Define the logger

In [3]:
log_file = settings.log_dir_path / f"cores_segmenation_{datetime.now():%Y-%m-%d_%H-%M-%S}.log"

logger.remove()
l = logger.add(lambda msg: print(msg, end=""))
l = logger.add(log_file, level="DEBUG", enqueue=True)

### Define cores for the analysis

In [5]:
core_dir = settings.analysis_dir / 'cores'
path_list = [core_dir / f for f in os.listdir(core_dir)]
path_list.sort()
path_list

[WindowsPath('C:/sdata_NRR-W/NRR-W_Analysis/cores/Core_000.zarr'),
 WindowsPath('C:/sdata_NRR-W/NRR-W_Analysis/cores/Core_001.zarr'),
 WindowsPath('C:/sdata_NRR-W/NRR-W_Analysis/cores/Core_002.zarr')]

### Setup

In [6]:
# setup builders of additional data elements

if getattr(settings,'additional_elements',None):
    
    builders_list = []

    for builder_settings in settings.additional_elements:
        
        params = dict(getattr(builder_settings,'parameters',None)) or {}

        builder = build_processor(builder_settings.category, builder_settings.type, **params) 
        
        builder_controller = ResourceBuildingController(builder=builder, 
                                            input_names=builder_settings.input, 
                                            output_names=builder_settings.output, 
                                            keep=builder_settings.keep, 
                                            overwrite=True,
                                            pyramid_levels=settings.sdata_storage.max_pyramid_level,
                                            downscale = settings.sdata_storage.downscale,
                                            chunk_size = settings.sdata_storage.chunk_size,
                                            )
        
        logger.info(f"Image transformer of type '{builder_settings.type}' for image '{builder_settings.input}' has been created.")

        builders_list.append(builder_controller)

else:
    builders_list = []
    logger.info("No resource builders specified.")

2025-11-10 10:51:10.801 | INFO     | __main__:<module>:23 - Image transformer of type 'normalize' for image 'DAPI' has been created.
2025-11-10 10:51:10.801 | INFO     | __main__:<module>:23 - Image transformer of type 'normalize' for image 'ECad' has been created.
Model fluorescence_nuclei_and_cells version 0.1.1 already downloaded in D:\instanseg\instanseg\utils\../bioimageio_models/, loading
Requesting default device: cuda
2025-11-10 10:51:12.510 | INFO     | __main__:<module>:23 - Image transformer of type 'instanseg' for image '['DAPI_norm', 'ECad_norm']' has been created.
2025-11-10 10:51:12.510 | INFO     | __main__:<module>:23 - Image transformer of type 'ring' for image '['instanseg_nucleus']' has been created.
2025-11-10 10:51:12.510 | INFO     | __main__:<module>:23 - Image transformer of type 'subtract' for image '['instanseg_cell', 'instanseg_nucleus']' has been created.


### Processing

In [7]:
# Optional - to detect problems early (runs <10s per sdata object)
for sd_path in path_list:
    
    logger.info(f"Validating {sd_path.name}")

    # get sdata
    sdata = sd.read_zarr(sd_path)

    # check that the pipeline can run on provide sdata
    settings.validate_pipeline(sdata)

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


2025-11-10 10:51:16.805 | INFO     | __main__:<module>:4 - Validating Core_000.zarr


version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


2025-11-10 10:51:19.939 | INFO     | multiplex_pipeline.utils.config_schema:validate_pipeline:188 - ✅ Pipeline validation successful.
2025-11-10 10:51:19.939 | INFO     | __main__:<module>:4 - Validating Core_001.zarr


version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


2025-11-10 10:51:22.079 | INFO     | multiplex_pipeline.utils.config_schema:validate_pipeline:188 - ✅ Pipeline validation successful.
2025-11-10 10:51:22.080 | INFO     | __main__:<module>:4 - Validating Core_002.zarr


version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


2025-11-10 10:51:24.656 | INFO     | multiplex_pipeline.utils.config_schema:validate_pipeline:188 - ✅ Pipeline validation successful.


In [ ]:
for sd_path in path_list:
    
    logger.info(f"Processing {sd_path.name}")

    # get sdata
    sdata = sd.read_zarr(sd_path)

    # check that the pipeline can run on provide sdata
    settings.validate_pipeline(sdata)

    # run builders of additional elements
    for builder_controller in builders_list:
        sdata = builder_controller.run(sdata)

version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


2025-11-10 10:51:25.956 | INFO     | __main__:<module>:3 - Processing Core_000.zarr


version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


2025-11-10 10:51:29.140 | INFO     | multiplex_pipeline.utils.config_schema:validate_pipeline:188 - ✅ Pipeline validation successful.
2025-11-10 10:51:29.140 | INFO     | multiplex_pipeline.processors.controller:validate_resolution_present:85 - All channels have required resolution level: 0
2025-11-10 10:52:23.185 | INFO     | multiplex_pipeline.processors.image_transformers:run:73 - Applied normalization (percentiles 1.0–99.8) → [0.0, 5750.0]
2025-11-10 10:52:23.412 | INFO     | multiplex_pipeline.processors.controller:run:212 - New element(s) '['DAPI_norm']' have been created.
2025-11-10 10:52:31.910 | INFO     | multiplex_pipeline.processors.controller:validate_resolution_present:85 - All channels have required resolution level: 0
2025-11-10 10:53:30.811 | INFO     | multiplex_pipeline.processors.image_transformers:run:73 - Applied normalization (percentiles 1.0–99.8) → [0.0, 478.0]
2025-11-10 10:53:30.979 | INFO     | multiplex_pipeline.processors.controller:run:212 - New element(s

### Sneak peek

In [9]:
np.max(sdata['instanseg_nucleus']['scale0']).compute()

<xarray.DataTree 'scale0'>
Group: /
    Dimensions:  ()
    Data variables:
        image    int64 8B 54912

In [5]:
# refresh the object
sdata_org = sd.read_zarr(path_list[0])
sdata_org

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requeste

SpatialData object, with associated Zarr store: C:\sdata_LCB004\LCB004-N-P_Analysis\cores\Core_000.zarr
├── Images
│     ├── 'AP2B': DataTree[cyx] (1, 19520, 17088), (1, 9760, 8544), (1, 4880, 4272)
│     ├── 'AR': DataTree[cyx] (1, 19520, 17088), (1, 9760, 8544), (1, 4880, 4272)
│     ├── 'CD45': DataTree[cyx] (1, 19520, 17088), (1, 9760, 8544), (1, 4880, 4272)
│     ├── 'CK14': DataTree[cyx] (1, 19520, 17088), (1, 9760, 8544), (1, 4880, 4272)
│     ├── 'CK818': DataTree[cyx] (1, 19520, 17088), (1, 9760, 8544), (1, 4880, 4272)
│     ├── 'DAPI': DataTree[cyx] (1, 19520, 17088), (1, 9760, 8544), (1, 4880, 4272)
│     ├── 'ECad': DataTree[cyx] (1, 19520, 17088), (1, 9760, 8544), (1, 4880, 4272)
│     ├── 'ER': DataTree[cyx] (1, 19520, 17088), (1, 9760, 8544), (1, 4880, 4272)
│     ├── 'FOXA1': DataTree[cyx] (1, 19520, 17088), (1, 9760, 8544), (1, 4880, 4272)
│     ├── 'GATA3': DataTree[cyx] (1, 19520, 17088), (1, 9760, 8544), (1, 4880, 4272)
│     ├── 'HER2': DataTree[cyx] (1, 19520, 170

In [ ]:
Interactive(sdata_org)

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\napari\utils\_proxies.py:103: FutureWarning: The `_dock_widgets` property is private and should not be used in any plugin code. Please use the `dock_widgets` property instead.
  return super().__getattr__(name)


2025-11-06 15:08:58.990 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-11-06 15:09:03.485 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\anndata\_core\aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
2025-11-06 15:09:59.363 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-11-06 15:09:59.372 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-11-06 15:09:59.375 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-11-06 15:18:54.127 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-11-06 15:18:54.135 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-11-06 15:18:59.090 | DEBUG    | napari_spatialdata.